In [1]:
# THIS IS THE SCRAPING OF THE DATA FROM THE WEBSITE
import requests
from bs4 import BeautifulSoup as bs4


In [2]:
base_url  = 'http://mlg.ucd.ie/modules/yalp'
result  = requests.get(base_url)
parsed_result = bs4(result.text)
#print(parsed_result)

In [4]:
# Make a list of dictionaries to have a recrod for each category
# Making dictionaries to support json format for easy processing
# next url endpoint is made adding endpoint to base url
category_data = [ { 'Category' : link.text.split(': ')[1], 'business_list_url' : '/'.join([base_url,link['href']])} for link in parsed_result.find_all('a')]

# print(category_data)

[{'Category': 'Automotive', 'business_list_url': 'http://mlg.ucd.ie/modules/yalp/automotive_list.html'}, {'Category': 'Bars', 'business_list_url': 'http://mlg.ucd.ie/modules/yalp/bars_list.html'}, {'Category': 'Health and medical', 'business_list_url': 'http://mlg.ucd.ie/modules/yalp/health_medical_list.html'}, {'Category': 'Hotels and travel', 'business_list_url': 'http://mlg.ucd.ie/modules/yalp/hotels_travel_list.html'}, {'Category': 'Restaurants', 'business_list_url': 'http://mlg.ucd.ie/modules/yalp/restaurants_list.html'}]


In [5]:
# get Data for next page - business list page : add this data to category data
# checking all pages one by one 
for record in category_data:
    business_data = []
    raw_data = requests.get(record.get('business_list_url'))
    bs4_obj = bs4(raw_data.text)
    for link in bs4_obj.find_all('a'):
        business_data.append({'reviews_url' : '/'.join([base_url, link.get('href')]),'business_name' :link.text })
    record['business_data'] = business_data


In [6]:
# this one going to the LAST page where we have the reviews - where reviews are present for a categlory 
import re
# get to next level i.e. reviews page for each business
for category_record in category_data:
    for business in category_record.get('business_data'):
        reviews_data = []
        reviews_url  = business.get('reviews_url')
        raw_data = requests.get(reviews_url)
        bs4_obj =  bs4(raw_data.text)
        for review in bs4_obj.find_all( "div", { "class" : "review"}):
            #details = review.find("p")
            #bs4_review_obj = bs4(review.text)
            review_details = {}
            for paragraph in review.find_all("p"):
                attribute = paragraph.get('class')[0]
                if attribute == 'review-top':
                    date = re.search('\d{4}-\d{2}-\d{2}',paragraph.text).group()
                    reviewer = paragraph.text.split('by ')[1]
                    review_details['date'] = date
                    review_details['reviewer'] = reviewer
                elif attribute == 'text':
                    review_details['text'] = paragraph.text
                elif attribute =='stars':
                    stars = paragraph.img.get('alt')
                    review_details['stars'] = stars
            reviews_data.append(review_details)
        #add review data to business for mapping
        business['reviews_data'] = reviews_data

In [12]:
# write all data in json file

import json
for data in category_data:
    category_name = data.get('Category')
    business_data = data.get('business_data')
    with open(f"./categories_reviews/{category_name}_reviews.json", 'w') as category_file:
        for business in business_data:
            for review in business.get("reviews_data"):
                category_file.write(json.dumps(review) + ',')
        print(f"written file for - {category_name}")

written file for - Automotive
written file for - Bars
written file for - Health and medical
written file for - Hotels and travel
written file for - Restaurants
